# Evaluate Langfuse LLM Traces with an External Evaluation Pipeline

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import openai
 
topic_suggestion = """ あなたは世界トップクラスのジャーナリストであり、
人々を最もワクワクさせる話題を見極めることに長けています。
あなたの仕事は、一般の人々が喜んで読むようなポップサイエンスのトピックを50個提案することです。
トピックが重複しないようにしてください。
出力はカンマ区切りのリストでなければなりません。リストを生成し、それ以外は何も生成しないでください。
数字の使用は禁止されています。
"""
 
output = openai.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": topic_suggestion
        }
    ],
    model="gpt-4o-mini",
 
    temperature=1
).choices[0].message.content
 
topics = [item.strip() for item in output.split(",")]
for topic in topics:
    print(topic)

ブラックホールの謎
人工知能の進化と未来
脳の可塑性と学習
地球外生命体の探求
サステナブルエネルギーの革新
クローン技術の可能性と倫理
バイオテクノロジーによる医療革命
確率論と日常生活
量子コンピューティングの未来
プラズマ物理学の奇跡
遺伝子編集技術CRISPRの可能性
ミクロの世界に生きる細菌の役割
人間の感情と脳の関係
宇宙の起源とビッグバン理論
サイボーグ技術の進化
感覚の不思議とその脳内メカニズム
環境変動と生態系の変化
ナノテクノロジーの応用
音の科学と音響技術
自閉症と脳の働き
心理学が教える幸せの秘訣
ゲノム解析と個別化医療
時間旅行の理論と可能性
スマートシティの未来像
脳波と外部機器のインターフェース
身体と心の関係: ヨガの科学
地球温暖化とその影響
未来の食糧問題とそれに対する解決策
ヒューマンエンハンスメントの倫理的側面
蜂の生態と人類の未来
データサイエンスとその社会的影響
フィルムと音楽の交差点: 脳の反応
エコロジカルフットプリントと持続可能性
人間の繁殖と進化の歴史
脳のリミットに挑戦するマインドフルネス
環境保護と心理学の関係
ヘルスケアテクノロジーの進化
感情と記憶の神秘
音楽が脳に与える影響
発明の歴史と偉大な発明家たち
人間の体内マイクロバイオームの秘密
宇宙の色とその科学
仮想現実とその心理的影響
知覚のトリック: 幻覚のメカニズム
進化論と人間の行動
風力発電の最新技術
過去から学ぶ未来の社会
AIによる芸術創造の可能性
脳科学が解き明かす創造性
人類の未来都市プロジェクト
自然界の驚くべき適応戦略
運動と精神の健康の関係


In [2]:
from langfuse.decorators import langfuse_context, observe
 
prompt_template = """
あなたは科学の専門家であり、複雑なトピックをわかりやすく説明できるコミュニケーション能力の持ち主です。
あなたの任務は、ユーザーの質問に魅力的で、有益で、親しみやすい方法で回答することです
魅力的で、有益で、親しみやすい方法で対応することです。事実を述べ、
専門用語は使用しないでください。回答は4文以内にしてください。
覚えておいてください。魅力的で楽しい回答を心がけてください!
 
Question: {question}
"""
 
@observe()
def explain_concept(topic):
    langfuse_context.update_current_trace(
        name=f"Explanation '{topic}'",
        tags=["ext_eval_pipelines"]
    )
    prompt = prompt_template.format(question=topic)
 
 
    return openai.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
 
        temperature=0.6
    ).choices[0].message.content
 
 
for topic in topics:
    print(f"Input: Please explain to me {topic.lower()}")
    print(f"Answer: {explain_concept(topic)} \n")

Input: Please explain to me ブラックホールの謎
Answer: ブラックホールは、宇宙の中でも特に不思議な存在です。とても強い重力を持っていて、光さえも逃げられないため、見ることができません。でも、周りの星やガスがその影響を受けるのを観察することで、ブラックホールの存在を確認できます。まるで宇宙の秘密を隠している魔法のようですね！ 

Input: Please explain to me 人工知能の進化と未来
Answer: 人工知能（AI）は、私たちの日常生活をどんどん便利にしてくれています。例えば、スマートフォンの音声アシスタントや、自動運転車などがその例です。未来には、もっと多くの分野でAIが活躍し、私たちの生活をサポートしてくれるでしょう。楽しみですね！ 

Input: Please explain to me 脳の可塑性と学習
Answer: 脳の可塑性は、脳が新しい情報を学んだり、経験に応じて変化したりする能力のことです。これにより、私たちは新しいスキルを身につけたり、記憶を強化したりできます。たとえば、楽器を演奏する練習をすると、脳の関連する部分がより活発になり、上達します。つまり、学ぶことで脳が成長し、私たちの能力も広がるんです！ 

Input: Please explain to me 地球外生命体の探求
Answer: 地球外生命体の探求は、私たちが宇宙の神秘を解き明かすためのワクワクする冒険です！科学者たちは、他の惑星や月の環境を調査し、生命の存在の可能性を探っています。例えば、火星やエウロパという氷の月には、生命が存在するかもしれない水があるんです。宇宙を旅する探査機たちが、私たちに未知の世界を見せてくれる日を楽しみにしましょう！ 

Input: Please explain to me サステナブルエネルギーの革新
Answer: サステナブルエネルギーの革新は、私たちの未来をより明るくする鍵です！太陽光や風力など、自然の力を使ってエネルギーを生み出す方法が進化し、より効率的で手頃になっています。これにより、環境への負担を減らしながら、私たちの生活を支えることができます。新しい技術のおかげで、持続可能なエネルギーがますます身近になってきていますね！ 

Input: Please explain to m

### 1. トレースを取得する

In [3]:
from langfuse import Langfuse
from datetime import datetime, timedelta
 
BATCH_SIZE = 10
TOTAL_TRACES = 50
 
langfuse = Langfuse()
 
now = datetime.now()
five_am_today = datetime(now.year, now.month, now.day, 5, 0)
five_am_yesterday = five_am_today - timedelta(days=1)
 
traces_batch = langfuse.fetch_traces(page=1,
                                     limit=BATCH_SIZE,
                                     tags="ext_eval_pipelines",
                                     from_timestamp=five_am_yesterday,
                                     to_timestamp=datetime.now()
                                   ).data
 
print(f"Traces in first batch: {len(traces_batch)}")

Traces in first batch: 10


### 2. 評価を実行する

2.1. カテゴリ評価

In [4]:
template_tone_eval = """
あなたは人間の感情的知性のエキスパートです。
 人間が書いたテキストのトーンを容易に特定できます。あなたの仕事は、<text/>のトーンを正確に特定することです
 テキストのトーンを正確に特定することです。 出力は、3つのトーンをカンマで区切ったリストです
 トーンのコンマ区切りのリストを作成します。リストのみを印刷し、他のものは印刷しないでください.
 
<possible_tones>
中立、自信、喜び、楽観、友好的、緊急、分析、尊敬
</possible_tones>
 
<example_1>
Input: 市民科学は、一般の人々を科学プロジェクトに参加させることで、研究に重要な役割を果たします
人々を科学プロジェクトに参加させることで、研究に重要な役割を果たしています。この協力により、研究者は
単独では収集不可能な膨大な量のデータを収集することができます。市民
科学者は、さまざまな分野における新たな発見や進歩につながる貴重な観察結果や洞察を提供しています
発見やさまざまな分野での進歩につながる。市民科学プロジェクトに参加することで、
科学プロジェクトに参加することで、個人は科学的研究に積極的に貢献
、身の回りの世界に対する理解に有意義な影響を与えることができます。.
 
Output: 尊敬、楽観、自信
</example_1>
 
<example_2>
Input: バイオニクスとは、生物学と工学を融合し、
人間の能力を向上させることができる機器を開発する分野です。人間と機械を融合させることで、
バイオニクスは、障害を持つ人々の生活の質を向上させたり、
、あるいは他の人のパフォーマンスを向上させることを目的としています。これらの技術は、しばしば
身体の自然なプロセスを模倣し、シームレスな統合を実現します。全体として、バイオニクスは
将来的に医療とテクノロジーに革命をもたらす大きな可能性を秘めています.
 
Output: 楽観的、自信に満ちている、分析的
</example_2>
 
<example_3>
Input: ソーシャルメディアは、精神衛生に良い影響も悪い影響も与える可能性があります。
。ポジティブな面では、人々がつながり、経験を共有し、
サポートを得ることもできます。しかし、ソーシャルメディアの過剰な使用は、
無力感、孤独感、不安感につながることもあります。
バランスを保ち、ソーシャルメディアが精神的な健康にどのような影響を与えるかを意識することが重要です。
時には休むことも、心の健康を優先させることも大切だということを覚えておいてください。
 
Output: 友好的、中立、敬意を払った
</example_3>
 
<text>
{text}
</text>
"""
 
 
test_tone_score = openai.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": template_tone_eval.format(
                text=traces_batch[1].output),
        }
    ],
    model="gpt-4o",
 
    temperature=0
).choices[0].message.content
print(f"User query: {traces_batch[1].input['args'][0]}")
print(f"Model answer: {traces_batch[1].output}")
print(f"Dominant tones: {test_tone_score}")

User query: 自然界の驚くべき適応戦略
Model answer: 自然界には驚くべき適応戦略がたくさんあります！たとえば、カメレオンは環境に溶け込むために色を変えることができ、捕食者から身を守ります。また、ペンギンは寒い南極での生活に特化した体型を持ち、仲間と一緒に体温を保ちながら泳ぎ回ります。これらの適応は、生命がどれほど創造的であるかを教えてくれますね！
Dominant tones: 喜び、尊敬、楽観


In [5]:
def tone_score(trace):
    return openai.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": template_tone_eval.format(text=trace.output),
            }
        ],
        model="gpt-4o-mini",
        temperature=0
    ).choices[0].message.content
 
tone_score(traces_batch[1])

'楽観、喜び、友好的'

2.2. 数値評価

In [7]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams, LLMTestCase
 
def joyfulness_score(trace):
		joyfulness_metric = GEval(
		    name="Correctness",
		    criteria="出力が魅力的で楽しいかどうかを判断する.",
		    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
		)
		test_case = LLMTestCase(
    input=trace.input["args"],
    actual_output=trace.output)
 
		joyfulness_metric.measure(test_case)
 
		print(f"Score: {joyfulness_metric.score}")
		print(f"Reason: {joyfulness_metric.reason}")
 
		return {"score": joyfulness_metric.score, "reason": joyfulness_metric.reason}
 
joyfulness_score(traces_batch[1])

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Score: 0.7337716937044428
Reason: The text is informative and highlights interesting adaptation strategies of animals, but it lacks a strong sense of amusement and creativity compared to more engaging narratives.


{'score': 0.7337716937044428,
 'reason': 'The text is informative and highlights interesting adaptation strategies of animals, but it lacks a strong sense of amusement and creativity compared to more engaging narratives.'}

### 3. スコアをLangfuseにプッシュする

In [8]:
langfuse.score(
    trace_id=traces_batch[1].id,
    name="tone",
    value=joyfulness_score(traces_batch[1])["score"],
    comment=joyfulness_score(traces_batch[1])["reason"]
)

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Score: 0.7429486221134627
Reason: The output is visually and content-wise engaging with examples of animal adaptations like chameleons and penguins. It is interesting and informative, making it enjoyable for readers interested in nature.


/Users/yoshidayasunori/Downloads/dev/LangChain_2/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Score: 0.7051072488020337
Reason: The output is informative and somewhat engaging, discussing unique adaptations in nature. However, it lacks strong visual appeal and does not fully immerse the reader.


### 4. すべてをまとめる

In [12]:
import math
 
for page_number in range(1, math.ceil(TOTAL_TRACES/BATCH_SIZE)):
 
    traces_batch = langfuse.fetch_traces(
        tags="ext_eval_pipelines",
        page=page_number,
        from_timestamp=five_am_yesterday,
        to_timestamp=five_am_today,
        limit=BATCH_SIZE
    ).data
 
    for trace in traces_batch:
        print(f"Processing {trace.name}")
 
        if trace.output is None:
            print(f"Warning: \n Trace {trace.name} had no generated output, \
            it was skipped")
            continue
 
        langfuse.score(
            trace_id=trace.id,
            name="tone",
            value=tone_score(trace)
        )
 
        jscore = joyfulness_score(trace)
        langfuse.score(
            trace_id=trace.id,
            name="joyfulness",
            value=jscore["score"],
            comment=jscore["reason"]
        )
 
    print(f"Batch {page_number} processed 🚀 \n")

Batch 1 processed 🚀 

Batch 2 processed 🚀 

Batch 3 processed 🚀 

Batch 4 processed 🚀 

